In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from utility.fetch_ import download_stock_data
ticker = ["tatasteel.ns"]
i=0
dd , dm = download_stock_data(ticker=ticker)
df = dd[:]


labelanswer = df[0:1]
df=df[1:]
df = df.sort_values('Date')
# Feature selection
data = df[['Open', 'High', 'Low']]

# Scaling the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Preparing training data
X_train = []
y_train_high = []
y_train_low = []

for i in range(60, len(scaled_data)):
    X_train.append(scaled_data[i-60:i, 0])  # Using Open prices for the last 60 days
    y_train_high.append(scaled_data[i, 1])  # High price as target
    y_train_low.append(scaled_data[i, 2])   # Low price as target

X_train = np.array(X_train)
y_train_high = np.array(y_train_high)
y_train_low = np.array(y_train_low)

# Reshaping for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Building the LSTM model
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=25))

# Two separate outputs for high and low prices
model.add(Dense(units=2))  

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, np.column_stack((y_train_high, y_train_low)), epochs=10, batch_size=32)
def predict_high_low(open_price, previous_data):
    # Create a copy of the previous data and add today's open price (without high and low yet)
    test_data = np.vstack((previous_data, [open_price, np.nan, np.nan]))

    # Scaling the data (ignore 'High' and 'Low' columns, as they are unknown for today's prediction)
    test_data_scaled = scaler.transform(test_data)

    # We only want the Open prices as input for prediction (the first column)
    test_data_scaled_input = test_data_scaled[-60:, 0].reshape(1, -1, 1)

    # Predict High and Low using the model
    predicted_high_low_scaled = model.predict(test_data_scaled_input)

    # Since the prediction output is scaled, we need to inverse transform it
    # We create a dummy array for inverse transformation
    predicted_high_low = scaler.inverse_transform([[open_price, predicted_high_low_scaled[0][0], predicted_high_low_scaled[0][1]]])

    # Return the predicted High and Low values
    return predicted_high_low[0][1], predicted_high_low[0][2]


c:\Users\korad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.4599
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1905
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0373
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0557
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0220
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0304
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0285
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0161
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0214
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0195


In [2]:
# Let's assume today's open price is 150.0
labelanswerdata = labelanswer.to_numpy()[0]
todays_open = labelanswerdata[1]

# Get the last 60 days' data (Open, High, Low) from the dataset
previous_data = df[['Open', 'High', 'Low']].values[-60:]

# Run the prediction
predicted_high, predicted_low = predict_high_low(todays_open, previous_data)

print(f"Predicted High: {predicted_high}")
print(f"Predicted Low: {predicted_low}")
print(labelanswer)


c:\Users\korad\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Predicted High: 158.67121996173955
Predicted Low: 154.62158689979242
           Date   Open        High         Low       Close   Adj Close  \
Date 2024-01-01  140.0  140.949997  139.050003  139.850006  137.087997   

        Volume  
Date  21186105  


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utility.fetch_ import download_stock_data
ticker = ["tatasteel.ns"]
i=0
dd , dm = download_stock_data(ticker=ticker)
df= dd[:]

# Sort by date in case the data isn't in order
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Separate label for today's data
labelanswer = df[0:1]
df = df[1:]

# Feature selection
data = df[['Open', 'High', 'Low']]

# Scaling the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Preparing training data
X_train = []
y_train_high = []
y_train_low = []

for i in range(60, len(scaled_data)):
    X_train.append(scaled_data[i-60:i, 0])  # Using Open prices for the last 60 days
    y_train_high.append(scaled_data[i, 1])  # High price as target
    y_train_low.append(scaled_data[i, 2])   # Low price as target

X_train = np.array(X_train)
y_train_high = np.array(y_train_high)
y_train_low = np.array(y_train_low)

# Reshaping for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Building the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=25))
model.add(Dense(units=2))  # Two separate outputs for high and low prices

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Early stopping to prevent overfitting (increase patience if needed)
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

# Train the model
model.fit(X_train, np.column_stack((y_train_high, y_train_low)), epochs=2000, batch_size=32, callbacks=[early_stopping])
model.fit(X_train, np.column_stack((y_train_high, y_train_low)), epochs=2000, batch_size=32, callbacks=[early_stopping])

# Prediction function
def predict_high_low(open_price, previous_data):
    # Append today's open price (High and Low are unknown)
    test_data = np.vstack((previous_data, [open_price, 0, 0]))  # Replace NaN with 0 for scaling

    test_data_scaled = scaler.transform(test_data)
    test_data_scaled_input = test_data_scaled[-60:, 0].reshape(1, -1, 1)
    
    # Predict High and Low
    predicted_high_low_scaled = model.predict(test_data_scaled_input)
    
    # Inverse scaling (replace dummy values with predicted ones in the High and Low columns)
    dummy_row = np.zeros((1, 3))  # 3 is for Open, High, and Low
    dummy_row[0, 0] = open_price  # First column is Open
    dummy_row[0, 1] = predicted_high_low_scaled[0][0]  # Second column is predicted High
    dummy_row[0, 2] = predicted_high_low_scaled[0][1]  # Third column is predicted Low

    predicted_high_low = scaler.inverse_transform(dummy_row)

    return predicted_high_low[0][1], predicted_high_low[0][2]  # Return predicted High and Low values

# Let's assume today's open price
labelanswerdata = labelanswer.to_numpy()[0]
todays_open = labelanswerdata[1]

# Get the last 60 days' data (Open, High, Low) from the dataset
previous_data = df[['Open', 'High', 'Low']].values[-60:]

# Run the prediction
predicted_high, predicted_low = predict_high_low(todays_open, previous_data)

# Compare with true values
true_high = labelanswerdata[2]
true_low = labelanswerdata[3]

print(f"Predicted High: {predicted_high} | True High: {true_high}")
print(f"Predicted Low: {predicted_low} | True Low: {true_low}")

# Calculate error metrics
if not np.isnan(predicted_high) and not np.isnan(predicted_low):
    mse_high = mean_squared_error([true_high], [predicted_high])
    mse_low = mean_squared_error([true_low], [predicted_low])
    mae_high = mean_absolute_error([true_high], [predicted_high])
    mae_low = mean_absolute_error([true_low], [predicted_low])

    print(f"MSE High: {mse_high}, MSE Low: {mse_low}")
    print(f"MAE High: {mae_high}, MAE Low: {mae_low}")
else:
    print("Invalid prediction: NaN encountered.")



Epoch 1/2000


c:\Users\korad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.4465
Epoch 2/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.4293
Epoch 3/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3979
Epoch 4/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3658
Epoch 5/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.3388
Epoch 6/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.3227
Epoch 7/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2800
Epoch 8/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2485
Epoch 9/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2105
Epoch 10/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1800
Epoch 11/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1543
Epoch 12/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1263
Epoch 13/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0973
Epoch 14/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0731
Epoch 15/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0504
Epoch 16/2000
4/4

KeyboardInterrupt: 